In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

ds = pd.read_csv("/kaggle/input/nasa-nearest-earth-objects/neo_v2.csv")

ds

In [ ]:
ds.info()

In [ ]:
ds.isna().sum()

In [ ]:
ds.describe().transpose()

# ***EDA***

In [ ]:
# fig, ax = plt.subplots(2, 3, figsize = (30, 10))
# for i, ax in zip(ds.select_dtypes(['int64', 'float64']).columns, ax.flat[:]):
#     sns.violinplot(x = ds[i], ax = ax)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize = (40, 10))
# for i, ax in zip(ds.select_dtypes('bool').columns, ax.flat[:]):
#     sns.histplot(x = ds[i].apply(lambda x: str(x)), ax = ax)

In [ ]:
sns.histplot(ds["orbiting_body"])

**Remove unused class**

In [ ]:
ds.info()

In [ ]:
ds.drop(["orbiting_body", "id", "name", "sentry_object"], axis = 1, inplace = True)

ds.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

ds["hazardous"] = LabelEncoder().fit_transform(ds["hazardous"])

ds.info()

In [ ]:
# fig, ax = plt.subplots(2, 3, figsize = (40, 20))
# for i, ax in zip(ds.columns[:-1], ax.flat[:]):
#     sns.scatterplot(data = ds, x = i, y = ds.columns[-1], hue = ds.columns[-1], ax = ax)

# sampling

In [ ]:
ds['hazardous'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

x_train, x_test, y_train, y_test = train_test_split(ds.iloc[:, : -1], ds.iloc[:, -1], train_size=.80)

x_train.shape, x_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [ ]:
print (y_train.value_counts(), '\n \n \n', y_test.value_counts())

In [ ]:
x_train, y_train = SMOTE().fit_resample(x_train, y_train)

In [ ]:
print (y_train.value_counts(), '\n \n \n', y_test.value_counts())

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)

y_pred = rfc.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print (classification_report(y_test, y_pred))

In [ ]:
def confusionMatrix(x, y):
    confusion = confusion_matrix(x, y)
    print (confusion, '\n \n Total : ', y.shape[0], '\n Truth : ', confusion[0, 0] + confusion[1, 1], '\n Error : ', confusion[0, 1] + confusion[1, 0])
    sns.heatmap(confusion,annot=True)

confusionMatrix(y_test, y_pred)

In [ ]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(verbose = False)

cbc.fit(x_train, y_train)

y_pred = cbc.predict(x_test)

In [ ]:
print (classification_report(y_pred, y_test))

print (" \n \n")

confusionMatrix(y_pred, y_test)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

xgb.fit(x_train, y_train)

y_pred = xgb.predict(x_test)

In [ ]:
print (classification_report(y_test, y_pred))

print ("\n")

confusionMatrix(y_test, y_pred)

In [ ]:
ds.info()

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.nn import relu, sigmoid
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.optimizers import Adam

model = models.Sequential([
    layers.Dense(5, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(384, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer = Adam(learning_rate = 0.0005
                              ), loss='binary_crossentropy', metrics=[Accuracy(name = 'accuracy'), Precision(name = 'precision'), Recall(name = 'recall')])

history = model.fit(x_train, y_train, epochs = 5)

print("\n \n")

model.evaluate(x_test, y_test)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:

y_pred = [
    1 if prob > 0.5 else 0 for prob in np.ravel(y_pred)
]

y_pred = np.array(y_pred)

In [ ]:
pd.Series(y_pred).value_counts()

In [ ]:
print (classification_report(y_test, y_pred))

print ("\n")

confusionMatrix(y_test, y_pred)